In [21]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
epochs = 2000
image_size = (360, 640)
batch_size = 16
learning_rate = 0.0001
momentum = 0.9

In [9]:
#Data preparation
class CustomImageDataset(Dataset):
    def __init__(self, labels_arr, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.DataFrame(labels_arr)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = torch.tensor(int(self.img_labels.iloc[idx, 1]))
        if self.transform:
            image = self.transform(image.float())
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
    
def create_dataset(folder, label):
    files = os.listdir(folder)
    for i in files:
        if i =='.DS_Store':
            files.remove(i)
    files = np.array(files)
    labels_arr = np.zeros(len(files), dtype=np.uint8)
    labels_arr.fill(label)
    labels_arr = np.column_stack((files, labels_arr))

    transform = transforms.Compose([transforms.Resize(image_size)])
    dataset = CustomImageDataset(labels_arr, folder, transform=transform)
    return dataset

In [10]:
#pure train datasets
dataset_on = create_dataset("/Users/gaky/Desktop/efir/on for use/on", int(0))
dataset_low = create_dataset("/Users/gaky/Desktop/efir/on for use/final low on", int(0))
dataset_off = create_dataset("/Users/gaky/Desktop/efir/off for use/off mix", int(1))
dataset_night = create_dataset("/Users/gaky/Desktop/efir/off for use/off night", int(2))


#agmented train datasets
dataset_off_changed = create_dataset("/Users/gaky/Desktop/efir/off for use/off changed", int(1))
dataset_off_blur = create_dataset("/Users/gaky/Desktop/efir/off for use/off blur", int(1))


#validation datasets
dataset_off_val = create_dataset("/Users/gaky/Desktop/efir/off for use/validation off mix", int(1))
dataset_low_val = create_dataset("/Users/gaky/Desktop/efir/on for use/low valid", int(0))
dataset_on_val = create_dataset("/Users/gaky/Desktop/efir/on for use/validation on", int(0))

train_dataset = ConcatDataset([dataset_on, dataset_low, dataset_off, dataset_night, dataset_off_blur])
print(len(train_dataset))

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloader_val_off = DataLoader(dataset_off_val, batch_size=len(dataset_off_val), shuffle=False)
dataloader_low_val = DataLoader(dataset_low_val, batch_size=len(dataset_low_val), shuffle=False)
dataloader_on_val = DataLoader(dataset_on_val, batch_size=len(dataset_on_val), shuffle=False)

dataloader_off = DataLoader(dataset_off, batch_size=len(dataset_off), shuffle=False)
dataloader_low = DataLoader(dataset_low, batch_size=len(dataset_low), shuffle=False)
dataloader_on = DataLoader(dataset_on, batch_size=len(dataset_on), shuffle=False)
dataloader_night = DataLoader(dataset_night, batch_size=len(dataset_night), shuffle=False)

850


In [11]:
import torch
import torch.nn as  nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
       

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x.clone()

        x = self.relu(self.batch_norm2(self.conv1(x)))
        x = self.batch_norm2(self.conv2(x))

        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        print(x.shape)
        print(identity.shape)
        x += identity
        x = self.relu(x)
        return x


        
        
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        print(x.shape)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)


In [12]:
model = ResNet(Bottleneck, [3,8,36,3], 3, 3).to(device)

# Loss and optimizer
loss_function = nn.CrossEntropyLoss()

optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.01, momentum = 0.9)
#optim = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1)

In [13]:
def save_model(epoch):
    model_scripted = torch.jit.script(model) # Export to TorchScript
    model_scripted.save("/Users/gaky/Desktop/efir/Models/strange model/"+ str(epoch) + ".pth")

In [14]:
def save_model2(epoch):
    torch.save(model.state_dict(), "/Users/gaky/Desktop/efir/Models/strange model2/"+ str(epoch) + ".pth")

In [18]:
def evaluate(dataloader, name):
    with torch.no_grad():
        dataset = iter(dataloader)
        x, y = next(dataset)
        
        x = x/255.
        x = x.to(device)
        y = y.to(device)
        model.eval()
        out = model(x.float())
        cat = torch.argmax(out, dim=1)
        
        if name == "off_val":
            for i in range(len(cat)):
                if cat[i] == 2:
                    cat[i] = 1

        accuracy = (cat == y).float().mean()
        print(accuracy, name)

In [19]:
def evaluate2(dataloader, name):
    with torch.no_grad():
        dick = []
        for i, (x, y) in enumerate(dataloader):
        
            x = x/255.
            x = x.to(device)
            y = y.to(device)
            model.eval()
            out = model(x.float())
            cat = torch.argmax(out, dim=1)


            if name == "off":
                for i in range(len(cat)):
                    if cat[i] == 2:
                        cat[i] = 1
            
            accuracy = (cat == y).float()
            dick.append(accuracy.mean())
        print(np.mean(dick), name)

In [23]:
losses = []
import time

for epoch in range(epochs):
    start = time.time()
    for i, (x, y) in enumerate(dataloader_train):
        x = x/255.
        x = x.to(device)
        y = y.to(device)
        model.train()
        
        model.zero_grad()
        start = time.time()
        
        out = model(x.float())
        
        loss = loss_function(out, y.long())
        loss_mean = loss.mean()
        loss_mean.backward()
        optim.step()

        print(epoch, i, loss_mean)
        
        losses.append(loss.detach().item())
        
        if epoch % 1 == 0 and epoch != 0 and i == 0:
            evaluate(dataloader_val_off, "off_val")
            evaluate(dataloader_low_val, "low_val")
            evaluate(dataloader_on_val, "on_val")
            
            #evaluate2(dataloader_on, "on")
            #evaluate2(dataloader_off, "off")
            #evaluate2(dataloader_low, "low")
            
            save_model2(epoch)
            #evaluate(dataloader_off, "off")
            #evaluate(dataloader_low, "low")
            #evaluate(dataloader_on, "on")
            #evaluate(dataloader_night, "night")
            
    stop = time.time()
    print(stop - start)
    print(np.mean(losses))

0 0 tensor(1.0511, grad_fn=<MeanBackward0>)
0 1 tensor(0.1308, grad_fn=<MeanBackward0>)
0 2 tensor(0.3343, grad_fn=<MeanBackward0>)
0 3 tensor(0.0895, grad_fn=<MeanBackward0>)
0 4 tensor(0.9416, grad_fn=<MeanBackward0>)
0 5 tensor(0.1095, grad_fn=<MeanBackward0>)
0 6 tensor(0.5079, grad_fn=<MeanBackward0>)
0 7 tensor(0.1917, grad_fn=<MeanBackward0>)
0 8 tensor(0.3785, grad_fn=<MeanBackward0>)
0 9 tensor(0.3109, grad_fn=<MeanBackward0>)
0 10 tensor(0.2796, grad_fn=<MeanBackward0>)
0 11 tensor(0.5198, grad_fn=<MeanBackward0>)
0 12 tensor(0.4670, grad_fn=<MeanBackward0>)
0 13 tensor(0.4765, grad_fn=<MeanBackward0>)
0 14 tensor(0.4806, grad_fn=<MeanBackward0>)
0 15 tensor(0.3925, grad_fn=<MeanBackward0>)
0 16 tensor(0.5978, grad_fn=<MeanBackward0>)
0 17 tensor(0.4283, grad_fn=<MeanBackward0>)
0 18 tensor(0.4230, grad_fn=<MeanBackward0>)
0 19 tensor(0.2079, grad_fn=<MeanBackward0>)
0 20 tensor(0.5202, grad_fn=<MeanBackward0>)
0 21 tensor(0.7179, grad_fn=<MeanBackward0>)
0 22 tensor(0.2463, 

KeyboardInterrupt: 